<a href="https://colab.research.google.com/github/Ryo-Tanohata/sig_beginner9/blob/main/beginner9_hist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
import gc
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
warnings.simplefilter('ignore')

 # 読み込むデータが格納されたディレクトリのパス，必要に応じて変更の必要あり
path = "/content/drive/My Drive/beginner9/"

train_df = pd.read_csv(path + "train.csv")
test_df = pd.read_csv(path + "test.csv")
submit = pd.read_csv(path + "sample_submission.csv", header=None)

# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['target']=-999

# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

# def eGFR(x):
#     if x['sex']==1:
#       return 194*(x['serum_creatinine']**-1.094)*(x['age']**-0.287)
#     else:
#       return 194*(x['serum_creatinine']**-1.094)*(x['age']**-0.287)*0.739

# all_df["eGFR"] = all_df.apply(eGFR, axis=1)

# 訓練データ、テストデータの分割
train_df = all_df[all_df['target']!=-999]
test_df = all_df[all_df['target']==-999]

y=train_df['target']
data=train_df.drop(['target','id'], axis=1)
test=test_df.drop(['target','id'], axis=1)

folds = 5
kf = StratifiedKFold(n_splits=folds,shuffle=True,random_state=0)

def lgbc2_action(data_, test_, y_, folds_):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])

    feats = [f for f in data_.columns]

    for i,(train_index,test_index) in enumerate(folds_.split(data_, y_)):
        X_cv_train = data_.iloc[train_index]
        X_cv_test = data_.iloc[test_index]
        y_cv_train = y_.iloc[train_index]
        y_cv_test = y_.iloc[test_index]

        model = HistGradientBoostingClassifier(loss='binary_crossentropy',max_iter=10001,learning_rate=0.1,random_state=0)
        model.fit(X_cv_train,y_cv_train)
        oof_preds[test_index] = model.predict(X_cv_test)
        sub_preds += model.predict_proba(test_)[:, 1] / folds_.n_splits
        
        print('Fold %2d ACC : %.6f' % (i + 1, accuracy_score(y_cv_test, oof_preds[test_index])))
        del model, X_cv_train, X_cv_test, y_cv_train, y_cv_test
        gc.collect()
        
    print('Full ACC score %.6f' % accuracy_score(y_, oof_preds)) 

    return oof_preds, sub_preds

oof_preds, test_preds = lgbc2_action(data, test, y, kf)

test_preds = pd.DataFrame(test_preds)
test_preds = test_preds.idxmax(axis=1)

submit[1] = test_preds
submit.to_csv(path + "submission.csv", index=False, header=False)

Fold  1 ACC : 0.870000
Fold  2 ACC : 0.830000
Fold  3 ACC : 0.870000
Fold  4 ACC : 0.890000
Fold  5 ACC : 0.855000
Full ACC score 0.863000
